One assumption I make about the ciphertext is that the letter corresponding to a break between words (space) is known:

In [1]:
import codecs
from collections import Counter

with codecs.open('mono_var08.KR', 'r', 'cp1251') as f:
    ciphertext = f.read()

space_sub = 'Ы'
ciphertext = ciphertext.replace(space_sub, ' ')
ciphertext_alphabet = ''.join(l for l, cnt in Counter(ciphertext).most_common() if l != ' ')

The most straightforward brute-force method I could think of is:

For each letter of the alphabet, create a list of possible permutations within a range $R$ based on occurrence frequency of letters in the ciphertext and the plaintext.

In [2]:
# Alphabet is sorted in most frequent -> least frequent letter
alphabet = 'ОЕАИНТСРВЛКМДПУЯЫЗЬЪБГЧЙХЖЮШЦЩЭФ'

R = 5
def permute(letter):
    i = ciphertext_alphabet.index(letter)
    return list(alphabet)[max(0, i - R):(i + R)]

permutations = {l: permute(l) for l in ciphertext_alphabet}
permutations

{'О': ['О', 'Е', 'А', 'И', 'Н'],
 'Г': ['О', 'Е', 'А', 'И', 'Н', 'Т'],
 'Л': ['О', 'Е', 'А', 'И', 'Н', 'Т', 'С'],
 'М': ['О', 'Е', 'А', 'И', 'Н', 'Т', 'С', 'Р'],
 'Р': ['О', 'Е', 'А', 'И', 'Н', 'Т', 'С', 'Р', 'В'],
 'Ф': ['О', 'Е', 'А', 'И', 'Н', 'Т', 'С', 'Р', 'В', 'Л'],
 'Т': ['Е', 'А', 'И', 'Н', 'Т', 'С', 'Р', 'В', 'Л', 'К'],
 'З': ['А', 'И', 'Н', 'Т', 'С', 'Р', 'В', 'Л', 'К', 'М'],
 'Ц': ['И', 'Н', 'Т', 'С', 'Р', 'В', 'Л', 'К', 'М', 'Д'],
 'Э': ['Н', 'Т', 'С', 'Р', 'В', 'Л', 'К', 'М', 'Д', 'П'],
 'Й': ['Т', 'С', 'Р', 'В', 'Л', 'К', 'М', 'Д', 'П', 'У'],
 'Х': ['С', 'Р', 'В', 'Л', 'К', 'М', 'Д', 'П', 'У', 'Я'],
 'Ш': ['Р', 'В', 'Л', 'К', 'М', 'Д', 'П', 'У', 'Я', 'Ы'],
 'Ю': ['В', 'Л', 'К', 'М', 'Д', 'П', 'У', 'Я', 'Ы', 'З'],
 'Ч': ['Л', 'К', 'М', 'Д', 'П', 'У', 'Я', 'Ы', 'З', 'Ь'],
 'К': ['К', 'М', 'Д', 'П', 'У', 'Я', 'Ы', 'З', 'Ь', 'Ъ'],
 'Ъ': ['М', 'Д', 'П', 'У', 'Я', 'Ы', 'З', 'Ь', 'Ъ', 'Б'],
 'П': ['Д', 'П', 'У', 'Я', 'Ы', 'З', 'Ь', 'Ъ', 'Б', 'Г'],
 'Щ': ['П', 'У', 'Я', 'Ы', 'З',

To constrain the search space a bit, I look at one-letter words:

In [4]:
letter_words = ['А', 'В', 'Ж', 'И', 'К', 'О', 'С', 'У', 'Э', 'Я']

def single_letter_permute(letter):
    return [l for l in permutations[letter] if l in letter_words]

single_perms = {w: single_letter_permute(w) for w in ciphertext.split(' ') if len(w) == 1}
updated_perms = {**permutations, **single_perms}
updated_perms

{'О': ['О', 'Е', 'А', 'И', 'Н'],
 'Г': ['О', 'Е', 'А', 'И', 'Н', 'Т'],
 'Л': ['О', 'Е', 'А', 'И', 'Н', 'Т', 'С'],
 'М': ['О', 'А', 'И', 'С'],
 'Р': ['О', 'Е', 'А', 'И', 'Н', 'Т', 'С', 'Р', 'В'],
 'Ф': ['О', 'Е', 'А', 'И', 'Н', 'Т', 'С', 'Р', 'В', 'Л'],
 'Т': ['Е', 'А', 'И', 'Н', 'Т', 'С', 'Р', 'В', 'Л', 'К'],
 'З': ['А', 'И', 'Н', 'Т', 'С', 'Р', 'В', 'Л', 'К', 'М'],
 'Ц': ['И', 'Н', 'Т', 'С', 'Р', 'В', 'Л', 'К', 'М', 'Д'],
 'Э': ['С', 'В', 'К'],
 'Й': ['Т', 'С', 'Р', 'В', 'Л', 'К', 'М', 'Д', 'П', 'У'],
 'Х': ['С', 'Р', 'В', 'Л', 'К', 'М', 'Д', 'П', 'У', 'Я'],
 'Ш': ['Р', 'В', 'Л', 'К', 'М', 'Д', 'П', 'У', 'Я', 'Ы'],
 'Ю': ['В', 'Л', 'К', 'М', 'Д', 'П', 'У', 'Я', 'Ы', 'З'],
 'Ч': ['Л', 'К', 'М', 'Д', 'П', 'У', 'Я', 'Ы', 'З', 'Ь'],
 'К': ['К', 'М', 'Д', 'П', 'У', 'Я', 'Ы', 'З', 'Ь', 'Ъ'],
 'Ъ': ['М', 'Д', 'П', 'У', 'Я', 'Ы', 'З', 'Ь', 'Ъ', 'Б'],
 'П': ['Д', 'П', 'У', 'Я', 'Ы', 'З', 'Ь', 'Ъ', 'Б', 'Г'],
 'Щ': ['П', 'У', 'Я', 'Ы', 'З', 'Ь', 'Ъ', 'Б', 'Г', 'Ч'],
 'Я': ['У', 'Я', 'Ы', 'З', '

In [ ]:
def iter_letter_permutations(letter_perms):
    letters = sorted(letter_perms.keys(), key=lambda l: len(letter_perms[l]))

    letter_i = 0
    perm_i = 0
    perm_i_stack = list()
    taken_letters = set()
    subs = dict()
    
    while letter_i >= 0:
        perms = letter_perms[letters[letter_i]]
        while perm_i < len(perms):
            letter_sub = perms[perm_i]
            if letter_sub in taken_letters:
                perm_i += 1
                continue
            taken_letters.add(letter_sub)
            subs[letters[letter_i]] = letter_sub
            if letter_i + 1 < len(letters):
                perm_i_stack.append(perm_i)
                letter_i += 1
                perm_i = 0
                break
            else:
                yield subs
                taken_letters.remove(letter_sub)
                perm_i += 1
        else:
            letter_i -= 1
            if letter_i >= 0:
                perm_i = perm_i_stack.pop()
                perms = letter_perms[letters[letter_i]]
                taken_letters.remove(perms[perm_i])
                perm_i += 1

In [ ]:
from ahocorapy.keywordtree import KeywordTree

natwords = [
    ' БЫ ', ' БУДТО ',
    ' ВЫ ', ' ВО ', ' ВЕСЬ ', ' ВСЕГДА ',
    ' ГДЕ ',
    ' ДА ', ' ДЛЯ ', ' ДО ', ' ДЕНЬ ',
    ' ЕЩЕ ',
    ' ЖЕ ',
    ' ЗА ',
    ' ИЛИ ', ' ИЗ ',
    ' КО ', ' КАК ', ' КОГДА ', ' КУДА ', ' КАКОЙ ', ' КАКАЯ ', ' КАКИЕ ', ' КОТОРЫЙ ', ' КОТОРАЯ ', ' КОТОРЫЕ ', ' КЕМ ',
    ' МЫ ', ' МЕЖДУ ',
    ' НА ', ' НЕ ', ' НО ', ' НУ ', ' НИ ', ' НАД ', ' НИКОГДА ', ' НИКАКОЙ ', ' НИКАКАЯ ', ' НИКАКИЕ ', ' НИКАКОГО ', ' НИКАКОЙ ', ' НИКАКИХ ', ' НОЧЬ ',
    ' ОБ ', ' ОН ', ' ОНА ', ' ОНИ ', ' ОНО ', ' ОТКУДА ', ' ОЧЕНЬ ',
    ' ПО ', ' ПОТОМ ', ' ПРО ',
    ' СО ',
    ' ТЫ ', ' ТОТ ',
    ' УЖ ', ' УЖЕ ', ' УТРО ',
    ' ХОТЬ ', ' ХОТЯ ',
    ' ЧТО ', ' ЧТОБЫ ', ' ЧЕРЕЗ ', ' ЧЕМ ',
    ' ЭТО ', ' ЭТОТ '
]
kwtree = KeywordTree(case_insensitive=False)
for w in natwords:
    kwtree.add(w)
kwtree.finalize()

def score_text(text):
    results = kwtree.search_all(text)
    if results is None:
        return 0
    score = 0
    for _ in results:
        score += 1
    return score

def brute_force(generate_from, generate_to, print_every):
    best = None
    generated = generate_from
    for subs in iter_letter_permutations(updated_perms):
        repl = encrypted.translate(subs)
        score = score_text(repl)
        if score > 0 and (best is None or best[0] < score):
            best = (score, repl)
        generated += 1
        if generated == generate_to:
            break
        if generated % print_every == 0:
            print(str(generated) + ': ' + str(best))

    print('Result from ' + str(generate_from) + ' to ' + str(generate_to) + ':')
    print(best)

## Results

I ran the following on a VPS with 8 cores (virtualized on first-gen Zen EPYC according to the provider):

In [ ]:
from multiprocessing import Pool

num_processes = 8
print_every = 1_000
generate_total = 8_000_000_000
generate_per_process = generate_total // num_processes

def run_brute_force(offset):
    brute_force(offset, offset + generate_per_process, print_every)

with Pool(num_processes) as pool:
    offsets = list(range(0, generate_total, generate_per_process))
    pool.map(run_brute_force, offsets)

pool.join()

After **twenty-four hours**, I barely scratched the surface of possible permutations:

```
234000000: None
4235700000: None
3233700000: None
6234300000: None
7230200000: None
5234000000: None
1237400000: None
```